In [1]:
import numpy as np

import json
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.models import *
from keras.layers import *
from keras.callbacks import *
from keras.optimizers import *
import keras.backend as K
import tensorflow as tf

/home/fabio/miniconda3/envs/dl/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
with open('data/sentipolc_char_index.json', 'r') as f:
    word_index = json.load(f)

In [3]:
with open('data/chars.tsv', 'w') as f:
    for w,_ in word_index.items():
        f.write('{}\n'.format(w))

In [4]:
data = np.load('data/sentipolc_char_seq.npz')

x_train = data['x_train']
y_train = data['y_train']

x_test = data['x_test']
y_test = data['y_test']

Check the max length of the text

In [5]:
max_len_seq = max([len(x) for x in x_train])
print('max len seq {}'.format(max_len_seq))
max_idx = max(word_index.values())
print('max id {}'.format(max_idx))

max len seq 190
max id 127


In [6]:
x_train_pad = sequence.pad_sequences(x_train, maxlen=max_len_seq)
x_test_pad = sequence.pad_sequences(x_test, maxlen=max_len_seq)

In [7]:
x_train_pad.shape

(4154, 190)

# Define metrics

In [8]:
def precision(y_true, y_pred):
    """Precision metric.

    Only computes a batch-wise average of precision.
     Computes the precision, a metric for multi-label classification of
     how many selected items are relevant.
      """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

In [9]:
def recall(y_true, y_pred):
    """Recall metric.
 
    Only computes a batch-wise average of recall.
    Computes the recall, a metric for multi-label classification of
    how many relevant items are selected.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

In [10]:
from sklearn.model_selection import StratifiedShuffleSplit

split_train_val = StratifiedShuffleSplit(1,test_size=0.2, random_state=45)

for train, test in split_train_val.split(x_train_pad, y_train):
    x_train_pad_split, y_train_pad_split = x_train_pad[train], y_train[train]
    x_val_pad_split, y_val_pad_split = x_train_pad[test], y_train[test]

In [11]:

callbacks = [
    ReduceLROnPlateau(patience=2, verbose=1),
    EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=1, mode='auto')
]

In [24]:
def build_model():
    model = Sequential()
    model.add(Embedding(input_dim=max_idx+1, output_dim=100, input_length=max_len_seq))
    model.add(GRU(32,  dropout=0.0, recurrent_dropout=0.0, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])
    
    return model

In [25]:
model = build_model()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 190, 100)          12800     
_________________________________________________________________
gru_1 (GRU)                  (None, 32)                12768     
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 33        
Total params: 25,601
Trainable params: 25,601
Non-trainable params: 0
_________________________________________________________________


In [26]:
history = model.fit(x_train_pad_split, y_train_pad_split, 
                    validation_data=(x_val_pad_split, y_val_pad_split),
                    batch_size=32, epochs=10, callbacks=callbacks)

Train on 3323 samples, validate on 831 samples
Epoch 1/10
3323/3323 [==============================] - 17s 5ms/step - loss: 0.6573 - acc: 0.6184 - val_loss: 0.6384 - val_acc: 0.6582
Epoch 2/10
3323/3323 [==============================] - 17s 5ms/step - loss: 0.6273 - acc: 0.6636 - val_loss: 0.6185 - val_acc: 0.6715
Epoch 3/10
3323/3323 [==============================] - 17s 5ms/step - loss: 0.6470 - acc: 0.6684 - val_loss: 0.6196 - val_acc: 0.6679
Epoch 4/10
3323/3323 [==============================] - 17s 5ms/step - loss: 0.6090 - acc: 0.6804 - val_loss: 0.6238 - val_acc: 0.6679
Epoch 5/10
3323/3323 [==============================] - 17s 5ms/step - loss: 0.6078 - acc: 0.6780 - val_loss: 0.6153 - val_acc: 0.6703
Epoch 6/10
3323/3323 [==============================] - 17s 5ms/step - loss: 0.6063 - acc: 0.6807 - val_loss: 0.6175 - val_acc: 0.6751
Epoch 7/10
3323/3323 [==============================] - 17s 5ms/step - loss: 0.5982 - acc: 0.6897 - val_loss: 0.6136 - val_acc: 0.6739
Epoch 8/

## Evaluate on the test set

In [ ]:
callbacks = [
    TensorBoard(histogram_freq=0, batch_size=128, embeddings_freq=1, embeddings_metadata=words),
]

In [ ]:
model = build_model()

history = model.fit(x_train_pad, y_train, batch_size=128, epochs=6, callbacks=callbacks)

In [ ]:
loss, acc = model.evaluate(x_test_pad, y_test)

In [ ]:
print('loss {}, acc {}'.format(loss, acc))

In [ ]:
#model.save('models/sentipolc_char_lstm.hdf5')

## Confusion matrix

In [ ]:
from sklearn import metrics

In [ ]:
predictions = model.predict_classes(x_train_pad, batch_size=128)

In [ ]:
tn, fp, fn, tp = metrics.confusion_matrix(y_train, predictions).ravel()

In [ ]:
recall = tp/(tp+fp)
precision = tp/(tp+fn)
f1 = metrics.f1_score(y_train,predictions)

print ('precision: {}'.format(precision), '\nrecall: {}'.format(recall), '\nf1: {}'.format(f1))